In [7]:
import pandas as pd
from datetime import datetime
from pathlib import Path

In [8]:
current_datetime =  datetime.now()
current_date_str = str(current_datetime.date()).replace('-','_')

current_directory = Path.cwd()
model_outputs_directory = current_directory / 'data/model_outputs' 

forwards_filename = current_date_str + '_forward_predictions.csv'
forwards_filepath = model_outputs_directory / forwards_filename

defense_filename = current_date_str + '_defense_predictions.csv'
defense_filepath = model_outputs_directory / defense_filename



forwards_df = pd.read_csv(forwards_filepath)
forwards_df = forwards_df.drop(columns=['Unnamed: 0'])
defense_df = pd.read_csv(defense_filepath)
defense_df = defense_df.drop(columns=['Unnamed: 0'])

In [9]:
forwards_df[forwards_df['Team'] == 'BOS']

,Player,Actual_Salary,Predicted_Salary,difference,Team
103,David Pastrnak,11250000.0,10888500.0,361498.22,BOS
104,Brad Marchand,6125000.0,6843974.0,-718973.66,BOS
105,Charlie Coyle,5250000.0,5458750.0,-208750.00,BOS
106,James van Riemsdyk,1000000.0,3006800.0,-2006800.00,BOS
107,Pavel Zacha,4750000.0,4713727.0,36272.61,BOS
108,Trent Frederic,2300000.0,2655458.0,-355458.35,BOS
109,Jake DeBrusk,4000000.0,4210250.0,-210250.01,BOS
110,Morgan Geekie,2000000.0,2536675.0,-536675.02,BOS
111,Danton Heinen,775000.0,2161060.0,-1386059.54,BOS
112,Matthew Poitras,870000.0,907125.0,-37125.00,BOS


In [10]:
defense_df[defense_df['Team'] == 'BOS']

,Player,Actual_Salary,Predicted_Salary,difference,Team
51,Charlie McAvoy,9500000.0,8440500.00,1059500.00,BOS
52,Hampus Lindholm,6500000.0,6370000.00,130000.00,BOS
53,Kevin Shattenkirk,1050000.0,1920685.83,-870685.83,BOS
54,Brandon Carlo,4100000.0,4124904.06,-24904.06,BOS
55,Matt Grzelcyk,3687500.0,3199666.66,487833.34,BOS
56,Derek Forbort,3000000.0,3035958.33,-35958.33,BOS
57,Parker Wotherspoon,775000.0,995023.82,-220023.82,BOS
